In [6]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load the dataset
data = pd.read_csv("/content/Entertainment_Recommendation_upd.csv", encoding='latin1')

# Extract user demographics
user_demographics = data[['User ID', 'Age', 'Gender']].drop_duplicates()

# One-hot encode Gender
encoder_gender = OneHotEncoder()
gender_encoded = encoder_gender.fit_transform(user_demographics[['Gender']]).toarray()

# Standardize Age
scaler_age = StandardScaler()
age_scaled = scaler_age.fit_transform(user_demographics[['Age']])

# Combine all user features into a single DataFrame
user_features = pd.DataFrame(np.hstack((gender_encoded, age_scaled)), index=user_demographics['User ID'])

# Create user-item matrix
user_item_matrix = pd.pivot_table(data, values='Rating', index='User ID', columns='Item ID', fill_value=0)

# Ensure user features and user-item matrix have the same set of users
common_users = user_features.index.intersection(user_item_matrix.index)
user_features = user_features.loc[common_users]
user_item_matrix = user_item_matrix.loc[common_users]

# Compute user similarities based on demographics
user_similarities = cosine_similarity(user_features)
print("Shape of User Similarity Matrix:", user_similarities.shape)

# Predict ratings using demographic similarities
def predict_ratings(user_item_matrix, user_similarities):
    denominator = np.sum(user_similarities, axis=1)
    numerator = np.dot(user_similarities, user_item_matrix)
    denominator_reshaped = denominator[:, np.newaxis]
    predicted_ratings = np.divide(numerator, denominator_reshaped, out=np.zeros_like(numerator), where=(denominator_reshaped != 0))
    return predicted_ratings

predicted_ratings = predict_ratings(user_item_matrix, user_similarities)

# Function to get item info (title, genre, type)
def get_item_info(item_id):
    item_data = data[data['Item ID'] == item_id]
    if not item_data.empty:
        title = item_data['Title'].values[0]
        genre = item_data['Genre'].values[0]
        item_type = item_data['Type'].values[0]  # Assuming there is a 'Type' column
        return title, genre, item_type
    return "Unknown Title", "Unknown Genre", "Unknown Type"

# Function to generate recommendations based on demographic information and filter by type
def generate_recommendations_for_demo(user_features, user_item_matrix, target_user_demo, desired_type, top_n=10):
    target_demo_sim = cosine_similarity([target_user_demo], user_features)[0]
    numerator = np.dot(target_demo_sim, user_item_matrix)
    denominator = np.sum(target_demo_sim)
    predicted_ratings_for_demo = numerator / denominator

    top_indices = np.argsort(predicted_ratings_for_demo)[::-1]

    top_items = []
    for i in top_indices:
        item_id = user_item_matrix.columns[i]
        title, genre, item_type = get_item_info(item_id)
        if item_type == desired_type:
            top_items.append((item_id, title, genre, item_type))
        if len(top_items) >= top_n:
            break

    return top_items

# Example target demographic: age=30, gender='Male'
target_demo = np.array([1, 0, 54])  # Assuming 0 is Female and 1 is Male after one-hot encoding

# Standardize the age feature
target_demo[2] = scaler_age.transform([[target_demo[2]]])[0, 0]

# Desired type: for example, 'Movie'
desired_type = 'Game'

# Generate recommendations
top_items = generate_recommendations_for_demo(user_features.values, user_item_matrix, target_demo, desired_type)
for i, (item_id, title, genre, item_type) in enumerate(top_items, 1):
    print(f"Recommendation {i}: Title: {title}, Genre: {genre}, Type: {item_type}")

Shape of User Similarity Matrix: (2157, 2157)
Recommendation 1: Title: Proactive human-resource function, Genre: Sandbox, Type: Game
Recommendation 2: Title: Function-based intangible standardization, Genre: Party, Type: Game
Recommendation 3: Title: Halo: Combat Evolved, Genre: First-Person Shooter, Type: Game
Recommendation 4: Title: Persevering homogeneous encoding, Genre: Horror, Type: Game
Recommendation 5: Title: Function-based regional adapter, Genre: Racing, Type: Game
Recommendation 6: Title: Profound human-resource model, Genre: Fighting, Type: Game
Recommendation 7: Title: Expanded discrete info-mediaries, Genre: Simulation, Type: Game
Recommendation 8: Title: Monitored value-added focus group, Genre: Educational, Type: Game
Recommendation 9: Title: Face-to-face asynchronous functionalities, Genre: Fighting, Type: Game
Recommendation 10: Title: Persistent responsive solution, Genre: Strategy, Type: Game


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
